In [204]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from bokeh.plotting import figure, show, output_file
from bokeh.palettes import brewer
from bokeh.models import ColumnDataSource
import bokeh

In [176]:
df = pd.read_csv('/home/andrew/Documents/ML/Cape_Town_water_levels/year_dam_data.csv', index_col='Year')
df = df.applymap(lambda x: x/100)
df

,berg_river,steenbras_lower,steenbras_upper,theewaterskloof,voëlvlei,wemmershoek
Year,,,,,,
2008,1.00,0.34,0.31,1.00,0.95,0.97
2009,1.00,0.97,1.00,1.00,0.94,0.95
2010,1.00,0.56,0.91,0.91,0.95,0.71
2011,0.85,0.65,0.97,0.80,0.80,0.72
2012,1.00,1.00,1.00,0.80,0.71,0.75
2013,0.78,0.68,0.78,0.85,0.68,0.75
2014,1.00,0.98,0.98,0.99,1.00,0.97
2015,0.78,0.75,0.84,0.63,0.45,0.60
2016,0.58,0.56,0.95,0.50,0.52,0.65


In [177]:
dam_max_capacity = {'berg_river': 130010,
               'steenbras_lower': 33517,
               'steenbras_upper': 31767,
               'theewaterskloof': 480188,
               'voëlvlei': 164095,
               'wemmershoek': 58644 }
for column in df:
    f = lambda x: x*(dam_max_capacity[column]/1000)
    df[column] = df[column].apply(f)
df

,berg_river,steenbras_lower,steenbras_upper,theewaterskloof,voëlvlei,wemmershoek
Year,,,,,,
2008,130.0100,11.39578,9.84777,480.18800,155.89025,56.88468
2009,130.0100,32.51149,31.76700,480.18800,154.24930,55.71180
2010,130.0100,18.76952,28.90797,436.97108,155.89025,41.63724
2011,110.5085,21.78605,30.81399,384.15040,131.27600,42.22368
2012,130.0100,33.51700,31.76700,384.15040,116.50745,43.98300
2013,101.4078,22.79156,24.77826,408.15980,111.58460,43.98300
2014,130.0100,32.84666,31.13166,475.38612,164.09500,56.88468
2015,101.4078,25.13775,26.68428,302.51844,73.84275,35.18640
2016,75.4058,18.76952,30.17865,240.09400,85.32940,38.11860


In [178]:
def stacked(df):
    df_top = df.cumsum(axis=1)
    df_bottom = df_top.shift(axis=1).fillna({df.columns[0]: 0})[::-1]
    df_stack = pd.concat([df_bottom, df_top], ignore_index=True)
    return df_stack
areas = stacked(df)

In [213]:
p = figure(title="Cape Town water levels", x_range=(2008, 2018), y_range=(0, 900))
p.grid.minor_grid_line_color = '#eeeeee'
p.ygrid.minor_grid_line_alpha = 0.05

In [ ]:
x2 = np.hstack((df.index[::-1], df.index))
source = ColumnDataSource(dict(
        xs = [x2] * areas.shape[1],
        ys = [areas[c].values for c in areas],
        color = brewer['PuBu'][areas.shape[1]],
        label = df.columns))
p.patches( xs='xs', ys='ys', color='color', legend='label', source=source)


In [ ]:
p.plot_height = 400
p.plot_width = 1000
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = '% Water'
p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

output_file('water_vis.html', title='Cape_Town')
show(p)